In [1]:
import numpy as np
import cv2

import os
import glob

In [2]:
DIM = (1280, 960)
K=np.array([[318.218395, 0.0, 640],
[0.000000, 319.006022, 480],
[0.000000, 0.000000, 1.000000]])
D=np.array([0.105789, -0.026515, 0.019449, -0.003727])

In [3]:
def undistort(img_path, balance=0.0, dim2=None, dim3=None):
    img = cv2.imread(img_path)
    dim1 = img.shape[:2][::-1]  #dim1 is the dimension of input image to un-distort    assert dim1[0]/dim1[1] == DIM[0]/DIM[1], "Image to undistort needs to have same aspect ratio as the ones used in calibration"    
    if not dim2: 
        dim2 = dim1    
    if not dim3:
        dim3 = dim1    
    scaled_K = K * dim1[0] / DIM[0]  # The values of K is to scale with image dimension.
    scaled_K[2][2] = 1.0  # Except that K[2][2] is always 1.0    
    # This is how scaled_K, dim2 and balance are used to determine the final K used to un-distort image. OpenCV document failed to make this clear!
    new_K = cv2.fisheye.estimateNewCameraMatrixForUndistortRectify(scaled_K, D, dim2, np.eye(3), balance=balance)
    map1, map2 = cv2.fisheye.initUndistortRectifyMap(scaled_K, D, np.eye(3), new_K, dim3, cv2.CV_16SC2)
    undistorted_img = cv2.remap(img, map1, map2, interpolation=cv2.INTER_LINEAR, borderMode=cv2.BORDER_CONSTANT)

    cv2.imwrite(img_path[:-4] + '_undistorted.jpg', undistorted_img)
    
    # cv2.imshow("undistorted", undistorted_img)
    # cv2.waitKey(0)
    # cv2.destroyAllWindows()

In [4]:
for i in np.arange(140, 401, 20):
    undistort('/home/shane/dataset100L/camera0_' + str(i) + '.jpg')

In [4]:
undistort('/home/shane/camera0.jpg')